In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
!pip install selenium
!pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [ ]:
# Function to get all the Company hrefs is a list
def get_comp_sites(url):
    
    # Parse the Page and create BS Object
    soup = BeautifulSoup(url, "html.parser")
    
    # Find the Section that has the listed Companies on the Page
    allSections = soup.find_all('section')
    for index, section in enumerate(allSections):
        if("Search Results" in str(section)):
            find_section = index

    # Find the Links to each Company    
    links = allSections[find_section].find_all('a')

    # There are extra links and Duplicates, so exclude the one's with HTTP and get rid of deuplicates
    hrefs = []
    for link in allSections[find_section].find_all('a'):
        if ('http' not in link):
            for i in link:            
                hrefs.append(link.get('href'))
    
    return hrefs

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Create Driver object from Service
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)


data = pd.DataFrame()
company_name = []

# Loop through each Page 
for j in range(1,261, 1):
    url = f'https://canada.constructconnect.com/dcn/construction-directory/search?ccpage={j}&perpage=16&phrase&category&location#results'
    driver.get(url)
    page = requests.get(f"https://canada.constructconnect.com/dcn/construction-directory/search?ccpage={j}&perpage=16&phrase&category&location#results").text
    sites = get_comp_sites(page)
    
#     Loop through and click on each of the 16 Companies, on each Page 
    for i,url in enumerate(hrefs):
        driver.get(f'https://canada.constructconnect.com/dcn/construction-directory/search?ccpage={j}&perpage=16&phrase&category&location#results')
        driver.find_element("xpath", f"//a[contains(@href, '{sites[i]}')]").click()

        # Get the Contact/Address info for Company (I used BeautifulSoup becasue it is more familiar to me)
        page = requests.get("https://canada.constructconnect.com" + hrefs[i])
        soup2 = BeautifulSoup(page.text)
        
#         Find the data from the Unorder List from 'company-detals' and get it's List Items
        for ulTag in soup2.find_all('ul', {'class': 'company-details'}):
            liTag = ulTag.find_all("li")
            if (liTag != []):
                data.append(liTag)
#         The Following 3 if statements extract the data (needed 3 different sized if statements because some are missing info)
        if len(liTag) == 4:
            work_org = liTag[0].text.strip()
            type_of_work = liTag[1].text.strip()     
            address = liTag[2].text.strip()
            phone_number = liTag[3].text.strip()  
            data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number}, ignore_index=True)

        if len(liTag) == 5:
            work_org = liTag[0].text.strip()
            type_of_work = liTag[1].text.strip()     
            address = liTag[2].text.strip()
            phone_number = liTag[3].text.strip()  
            website = liTag[4].text.strip()
            data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number, "Website": website}, ignore_index=True)


        if len(liTag) == 6:
            work_org = liTag[0].text.strip()
            type_of_work = liTag[1].text.strip()     
            address = liTag[2].text.strip()
            phone_number = liTag[3].text.strip()  
            website = liTag[4].text.strip()
            email = liTag[5].text.strip()
            data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number, "Website": website, "Email": email}, ignore_index=True)

        # Get Company Name (will add to final data later)
        compName = soup2.select('h1.company-name')[0].text.strip()
        company_name.append(compName)

    


In [1471]:
new_data = data
new_data = pd.DataFrame(0.0, index=np.arange(len(data)), columns=["Work_Org", "Type_of_Work", "Address", "Phone_Number", "Website", "Email"])

# for loop to Clean Data
x = len(data)
for i in range(0,x,1):
    for j in range(0,6,1):
        
        # Find Type of Org
        new_data.iloc[i,0] = data.iloc[i,0]
    
        # Find Type of Work
        if ('\n' not in str(data.iloc[i,1])):
            new_data.iloc[i,1] = data.iloc[i,1]
        
        # Find Address - the address has multiple \n tags
        x = len(re.findall('\n', str(data.iloc[i,j])))
        if x > 1:
            new_data.iloc[i,2] = data.iloc[i,j]
        
        # Find Phone Number - search for more then 8 digits
        count = 0
        y = str(data.iloc[i,j])
        for a in y:
            if a.isdigit():
                count = count + 1
        if count > 8:
            new_data.iloc[i,3] = data.iloc[i,j]
        
        # Find Website
        if ('@' not in str(data.iloc[i,j])) and ('.com' in str(data.iloc[i,j])) or ('.ca' in str(data.iloc[i,j])) and ('@' not in str(data.iloc[i,j])):
            new_data.iloc[i,4] = data.iloc[i,j]
            
        # Find Email - emails have the @ sign
        if '@' in str(data.iloc[i,j]):
            new_data.iloc[i,5] = data.iloc[i,j]

new_data


,Work_Org,Type_of_Work,Address,Phone_Number,Website,Email
0,Sub-Contractors,Thermal and Moisture Protection,"32 Goodmark Place Unit 17, \n Etobicoke, \n...",416-679-8911,0.0,0.0
1,General Contractors,General Building,"27 Nashville Ave, Suite 1, \n York, \n O...",416-825-9037\nFax (888) 521-3334,0.0,0.0
2,General Contractors,Civil,"1160 Waterdown Rd Unit 1, \n Burlington, \n...",905-633-6938\nFax 905-633-7132,0.0,0.0
3,Sub-Contractors,"Heating, Ventilating, and Air Conditioning (HV...","2285 St-Laurent Blvd. Unit B19, \n Ottawa, ...",613-832-4595\nFax 613-260-3540,3vmechanical.ca/,info@3vmechanical.ca
4,Sub-Contractors,Exterior Improvements,"5104 Jones Baseline, \n Guelph, \n Ontar...",519-221-8688,0.0,0.0
5,General Contractors,General Building,"34 allanhurst Crescent, \n Brampton, \n ...",416-732-1966,0.0,0.0
6,Construction Software,0.0,"3760 14th Avenue, \n Markham, \n Ontario...",800-465-6475\nFax 800-947-7966,canada.constructconnect.com,customerrelationscanada@constructconnect.com
7,Sub-Contractors,Electronic Safety and Security,"360 Carlingview Dr, \n Etobicoke, \n Ont...",416-798-2228\nFax (416) 798-3962,www.360asc.com/,info@360asc.com
8,"Site Services, Sub-Contractors",Earthwork,"815 Great Northern Road, \n Sault Ste Marie...",705-253-3569\nFax 705-253-8807,0.0,shanecorbett@shaw.ca
9,"General Contractors, Site Services",General Building,"2355 Royal Windsor Dr, Unit 6, \n Mississau...",905-828-6868\nFax 905-855-1744,www.aandocontracting.com,info@aandocontracting.com


In [1472]:
# Seperate Address to Clean and Create muliple columns for it
address_fix = new_data[:]
address_fix.drop(address_fix.columns.difference(['Address']), 1, inplace=True)
address_fix.head()

,Address
0,"32 Goodmark Place Unit 17, \n Etobicoke, \n..."
1,"27 Nashville Ave, Suite 1, \n York, \n O..."
2,"1160 Waterdown Rd Unit 1, \n Burlington, \n..."
3,"2285 St-Laurent Blvd. Unit B19, \n Ottawa, ..."
4,"5104 Jones Baseline, \n Guelph, \n Ontar..."


In [1473]:
# Create 4 columns for Address
new = address_fix["Address"].str.split("\n", n = 3, expand = True)
new = new.apply(lambda x: x.str.replace(',', ''))
new.columns=["Street", "City", "State", "Zip Code"]
new.head()

,Street,City,State,Zip Code
0,32 Goodmark Place Unit 17,Etobicoke,Ontario,M9W 6J4
1,27 Nashville Ave Suite 1,York,Ontario,M6M 1J2
2,1160 Waterdown Rd Unit 1,Burlington,Ontario,L7T 4A8
3,2285 St-Laurent Blvd. Unit B19,Ottawa,Ontario,K1G 4Z5
4,5104 Jones Baseline,Guelph,Ontario,N1H 6H8


In [1474]:
# Combine Address back to the data
result = pd.concat([new_data, new], axis=1)
result = result.drop('Address', 1)
result.head()

,Work_Org,Type_of_Work,Phone_Number,Website,Email,Street,City,State,Zip Code
0,Sub-Contractors,Thermal and Moisture Protection,416-679-8911,0.0,0.0,32 Goodmark Place Unit 17,Etobicoke,Ontario,M9W 6J4
1,General Contractors,General Building,416-825-9037\nFax (888) 521-3334,0.0,0.0,27 Nashville Ave Suite 1,York,Ontario,M6M 1J2
2,General Contractors,Civil,905-633-6938\nFax 905-633-7132,0.0,0.0,1160 Waterdown Rd Unit 1,Burlington,Ontario,L7T 4A8
3,Sub-Contractors,"Heating, Ventilating, and Air Conditioning (HV...",613-832-4595\nFax 613-260-3540,3vmechanical.ca/,info@3vmechanical.ca,2285 St-Laurent Blvd. Unit B19,Ottawa,Ontario,K1G 4Z5
4,Sub-Contractors,Exterior Improvements,519-221-8688,0.0,0.0,5104 Jones Baseline,Guelph,Ontario,N1H 6H8


In [1475]:
# Add the Company Name to the rest of the data
company_name = pd.DataFrame(company_name)
final = pd.concat([result, company_name], axis =1)

In [1476]:
# first_column = final2.pop(0)
# final2.insert(0, '0', first_column)
final.rename(columns={'0': "Company Name"}, inplace=True)



,Work_Org,Type_of_Work,Phone_Number,Website,Email,Street,City,State,Zip Code,0
0,Sub-Contractors,Thermal and Moisture Protection,416-679-8911,0.0,0.0,32 Goodmark Place Unit 17,Etobicoke,Ontario,M9W 6J4,1867832 Ontario Inc
1,General Contractors,General Building,416-825-9037\nFax (888) 521-3334,0.0,0.0,27 Nashville Ave Suite 1,York,Ontario,M6M 1J2,2580328 Ontario Inc o/a MIG Underground
2,General Contractors,Civil,905-633-6938\nFax 905-633-7132,0.0,0.0,1160 Waterdown Rd Unit 1,Burlington,Ontario,L7T 4A8,A Cosmos Concrete & Paving
3,Sub-Contractors,"Heating, Ventilating, and Air Conditioning (HV...",613-832-4595\nFax 613-260-3540,3vmechanical.ca/,info@3vmechanical.ca,2285 St-Laurent Blvd. Unit B19,Ottawa,Ontario,K1G 4Z5,3V Mechanical Inc
4,Sub-Contractors,Exterior Improvements,519-221-8688,0.0,0.0,5104 Jones Baseline,Guelph,Ontario,N1H 6H8,39 Seven Inc
5,General Contractors,General Building,416-732-1966,0.0,0.0,34 allanhurst Crescent,Brampton,Ontario,L6P 1C6,2490179 Ontario Limited (MMA Construction)
6,Construction Software,0.0,800-465-6475\nFax 800-947-7966,canada.constructconnect.com,customerrelationscanada@constructconnect.com,3760 14th Avenue,Markham,Ontario,L3R 3T7,ConstructConnect
7,Sub-Contractors,Electronic Safety and Security,416-798-2228\nFax (416) 798-3962,www.360asc.com/,info@360asc.com,360 Carlingview Dr,Etobicoke,Ontario,M9W 5X9,360 Advanced Security Corporation
8,"Site Services, Sub-Contractors",Earthwork,705-253-3569\nFax 705-253-8807,0.0,shanecorbett@shaw.ca,815 Great Northern Road,Sault Ste Marie,Ontario,P6A 5K7,1187839 Ontario Ltd
9,"General Contractors, Site Services",General Building,905-828-6868\nFax 905-855-1744,www.aandocontracting.com,info@aandocontracting.com,2355 Royal Windsor Dr Unit 6,Mississauga,Ontario,L5J 4S8,A & O Contracting Inc


In [1468]:
final

,Company Name,Work_Org,Type_of_Work,Phone_Number,Website,Email,Street,City,State,Zip Code
0,1867832 Ontario Inc,Sub-Contractors,Thermal and Moisture Protection,416-679-8911,0.0,0.0,32 Goodmark Place Unit 17,Etobicoke,Ontario,M9W 6J4
1,2580328 Ontario Inc o/a MIG Underground,General Contractors,General Building,416-825-9037\nFax (888) 521-3334,0.0,0.0,27 Nashville Ave Suite 1,York,Ontario,M6M 1J2
2,A Cosmos Concrete & Paving,General Contractors,Civil,905-633-6938\nFax 905-633-7132,0.0,0.0,1160 Waterdown Rd Unit 1,Burlington,Ontario,L7T 4A8
3,3V Mechanical Inc,Sub-Contractors,"Heating, Ventilating, and Air Conditioning (HV...",613-832-4595\nFax 613-260-3540,3vmechanical.ca/,info@3vmechanical.ca,2285 St-Laurent Blvd. Unit B19,Ottawa,Ontario,K1G 4Z5
4,39 Seven Inc,Sub-Contractors,Exterior Improvements,519-221-8688,0.0,0.0,5104 Jones Baseline,Guelph,Ontario,N1H 6H8
5,2490179 Ontario Limited (MMA Construction),General Contractors,General Building,416-732-1966,0.0,0.0,34 allanhurst Crescent,Brampton,Ontario,L6P 1C6
6,ConstructConnect,Construction Software,0.0,800-465-6475\nFax 800-947-7966,canada.constructconnect.com,customerrelationscanada@constructconnect.com,3760 14th Avenue,Markham,Ontario,L3R 3T7
7,360 Advanced Security Corporation,Sub-Contractors,Electronic Safety and Security,416-798-2228\nFax (416) 798-3962,www.360asc.com/,info@360asc.com,360 Carlingview Dr,Etobicoke,Ontario,M9W 5X9
8,1187839 Ontario Ltd,"Site Services, Sub-Contractors",Earthwork,705-253-3569\nFax 705-253-8807,0.0,shanecorbett@shaw.ca,815 Great Northern Road,Sault Ste Marie,Ontario,P6A 5K7
9,A & O Contracting Inc,"General Contractors, Site Services",General Building,905-828-6868\nFax 905-855-1744,www.aandocontracting.com,info@aandocontracting.com,2355 Royal Windsor Dr Unit 6,Mississauga,Ontario,L5J 4S8


In [ ]:
final.to_csv(C:\Users\djoco\UpWork\Const_Web_Scrap, encoding='utf-8', index=False)